In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXi 78BbewkjRBfeT3BlbkFJoLeEVAbOl0kiGIZTW WikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
complexity_based_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Complexity-based Prompting for complex reasoning tasks.
    
    Complexity-based Prompting is an advanced form of Chain-of-Thought (CoT) that focuses on complex examples and prioritizes more elaborate reasoning chains. 
    
    This technique significantly improves performance on mathematical and complex reasoning tasks by:
    1. Selecting and analyzing complex examples that require multiple reasoning steps
    2. Generating thorough reasoning chains that explore the problem space comprehensively
    3. Prioritizing more detailed explanations, under the premise that longer reasoning indicates higher quality answers
    
    The Complexity-based Prompting process includes the following steps:
    1. Problem complexity assessment: Evaluate the complexity of the problem based on length, required reasoning steps, and conceptual depth.
    2. Detailed reasoning chain: Develop an extensive, step-by-step reasoning process that thoroughly addresses all components of the problem.
    3. Alternative pathway consideration: Consider multiple approaches or solution paths to verify the answer.
    4. Final answer with justification: Provide a final answer with clear justification based on the most reliable reasoning chain.
    
    Please structure your response in the following format:
    
    Step 1: [Problem complexity assessment]
    Step 2: [Detailed reasoning chain]
    Step 3: [Alternative pathway consideration]
    Step 4: [Final answer with justification]
    
    Example:
    Question: "A delivery company charges $10 for the first kg and $7 for each additional kg. If a package costs $80 to ship, how many kg does it weigh?"
    
    Step 1: Problem complexity assessment:
    This is a multi-step algebraic word problem that requires:
    - Understanding the pricing structure (different rates for first vs. additional kg)
    - Setting up and solving an equation with variables
    - Checking if the solution makes sense in the context
    This problem has moderate complexity requiring at least 3-4 distinct reasoning steps.
    
    Step 2: Detailed reasoning chain:
    Let me denote the total weight as x kg.
    
    Given information:
    - First kg costs $10
    - Each additional kg costs $7
    - Total shipping cost is $80
    
    Let's set up an equation:
    - For x kg, we have 1 kg at $10 and (x-1) kg at $7 each
    - This gives us: 10 + 7(x-1) = 80
    
    Let's solve this equation:
    - 10 + 7x - 7 = 80
    - 7x + 3 = 80
    - 7x = 77
    - x = 77/7 = 11
    
    So according to this calculation, the package weighs 11 kg.
    
    Step 3: Alternative pathway consideration:
    Let me verify this result by calculating the cost for an 11 kg package:
    - First kg: $10
    - Additional 10 kg: 10 × $7 = $70
    - Total: $10 + $70 = $80
    
    This matches our given cost of $80.
    
    Let me also consider a different approach:
    - If the package weighed x kg, then the cost would be: 10 + 7(x-1)
    - We can rewrite this as: 10 + 7x - 7 = 3 + 7x
    - Setting this equal to $80: 3 + 7x = 80
    - Subtracting 3: 7x = 77
    - Dividing by 7: x = 11
    
    Both approaches yield the same result, confirming our answer.
    
    Step 4: The package weighs 11 kg. This answer is justified because:
    - The mathematical equation 10 + 7(x-1) = 80 yields x = 11
    - Verification shows that an 11 kg package would indeed cost $80 ($10 for the first kg plus $70 for the remaining 10 kg)
    - An alternative solution approach confirms the same result
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: Problem complexity assessment:
    [Evaluate the complexity of the problem by identifying:
    - The number of variables, constraints, or relationships involved
    - The number of distinct reasoning steps required
    - The conceptual knowledge needed to solve the problem
    - Any potential subtleties or edge cases that need consideration]
    
    Step 2: Detailed reasoning chain:
    [Provide an extensive, step-by-step reasoning process that addresses all aspects of the problem:
    1. [Clear statement of the approach and initial setup]
    2. [Systematic development of the solution with careful attention to each step]
    3. [Explicit calculations or logical deductions shown in detail]
    4. [Continued detailed reasoning until reaching a provisional answer]
    ...]
    
    Step 3: Alternative pathway consideration:
    [Explore at least one alternative approach or verification method:
    - [Verification of the answer by working backwards]
    - [Alternative solution method that approaches the problem differently]
    - [Discussion of how the different approaches converge on the same answer or why one approach is more reliable]]
    
    Step 4: [Provide a clear, precise answer to the original question, with explicit justification based on the most reliable reasoning chain and verification process]
    """)
])

In [ ]:
complexity_based_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Complexity-based Prompting examples based on user questions.
    
    Complexity-based Prompting is a specialized technique within Chain-of-Thought (CoT) prompting that:
    1. Selects complex examples for annotation based on factors like question length or reasoning steps required
    2. Includes these complex examples in the prompt to guide reasoning
    3. During inference, samples multiple reasoning chains (answers)
    4. Uses a majority vote among chains exceeding a certain length threshold
    5. Operates under the premise that longer reasoning indicates higher answer quality
    
    This approach has shown improvements on mathematical reasoning datasets by focusing on complex examples and leveraging multiple reasoning paths.
    
    Please structure your response in the following format:
    
    Step 1: Complexity Analysis
    [Analyze the complexity of the question and identify key factors like length, number of reasoning steps, etc.]
    
    Step 2: Complex Exemplars
    [Provide 3-5 exemplars of similar complexity with detailed reasoning chains]
    
    Step 3: Multiple Reasoning Paths
    [Generate 3 different reasoning paths/approaches to solve the problem]
    
    Step 4: Length-Based Filtering
    [Identify which reasoning chains exceed the complexity threshold (e.g., number of steps)]
    
    Step 5: Majority Vote Solution
    [Determine the final answer based on majority vote among the qualified reasoning chains]
    
    Example:
    Question: "A store sells notebooks for $4.50 each and pens for $2.25 each. If a customer buys 3 notebooks and twice as many pens, and pays with a $50 bill, how much change will they receive?"
    
    Complexity Analysis:
    This question involves multiple steps: calculating the cost of notebooks, calculating the cost of pens, finding the total cost, and determining change from a payment. The question length is moderate, and it requires 4-5 reasoning steps, making it a good candidate for Complexity-based Prompting.
    
    Complex Exemplars:
    Exemplar 1: 
    Q: If a shirt costs $35 and pants cost $50, how much would a person pay for 2 shirts and 3 pants with a 15% discount on the total?
    A: First, I'll calculate the cost of the shirts.
    Cost of shirts = 2 × $35 = $70
    Next, I'll calculate the cost of the pants.
    Cost of pants = 3 × $50 = $150
    Subtotal = $70 + $150 = $220
    To find the discount amount, I calculate 15% of the subtotal.
    Discount = 0.15 × $220 = $33
    Final cost = $220 - $33 = $187
    Therefore, the person would pay $187.
    
    Exemplar 2:
    Q: A car travels at 60 mph for 2 hours, then at 50 mph for 3 hours. What is the average speed for the entire trip?
    A: First, I need to find the total distance traveled.
    Distance at 60 mph = 60 miles/hour × 2 hours = 120 miles
    Distance at 50 mph = 50 miles/hour × 3 hours = 150 miles
    Total distance = 120 miles + 150 miles = 270 miles
    Next, I need to find the total time.
    Total time = 2 hours + 3 hours = 5 hours
    Average speed = Total distance ÷ Total time = 270 miles ÷ 5 hours = 54 mph
    Therefore, the average speed for the entire trip is 54 mph.
    
    Exemplar 3:
    Q: A recipe requires 2/3 cup of flour per dozen cookies. If I want to make 30 cookies, how much flour do I need?
    A: First, I need to find how many dozen cookies I'm making.
    Number of cookies = 30
    Number of dozen = 30 ÷ 12 = 2.5 dozen
    Now I can find the amount of flour needed.
    Flour needed = 2/3 cup × 2.5 dozen = 2/3 × 2.5 = 5/3 cups = 1 and 2/3 cups
    Therefore, I need 1 and 2/3 cups of flour.
    
    Multiple Reasoning Paths:
    Reasoning Path 1:
    Step 1: Calculate the cost of notebooks.
    Cost of notebooks = 3 × $4.50 = $13.50
    Step 2: Determine how many pens the customer buys.
    Number of pens = 2 × 3 = 6 pens
    Step 3: Calculate the cost of pens.
    Cost of pens = 6 × $2.25 = $13.50
    Step 4: Find the total cost.
    Total cost = $13.50 + $13.50 = $27.00
    Step 5: Calculate the change.
    Change = $50.00 - $27.00 = $23.00
    Therefore, the customer will receive $23.00 in change.
    
    Reasoning Path 2:
    Step 1: Determine how many pens the customer buys.
    Number of pens = 2 × 3 notebooks = 6 pens
    Step 2: Calculate the total cost of all items.
    Cost of 3 notebooks at $4.50 each = 3 × $4.50 = $13.50
    Cost of 6 pens at $2.25 each = 6 × $2.25 = $13.50
    Total purchase amount = $13.50 + $13.50 = $27.00
    Step 3: Calculate the change from $50.
    Change = $50.00 - $27.00 = $23.00
    Therefore, the customer will receive $23.00 in change.
    
    Reasoning Path 3:
    Step 1: Set up an equation for the total cost.
    Total cost = (Cost per notebook × Number of notebooks) + (Cost per pen × Number of pens)
    Total cost = ($4.50 × 3) + ($2.25 × (2 × 3))
    Step 2: Simplify the equation.
    Total cost = $13.50 + $2.25 × 6
    Total cost = $13.50 + $13.50
    Total cost = $27.00
    Step 3: Calculate the change.
    Change = Amount paid - Total cost
    Change = $50.00 - $27.00
    Change = $23.00
    Therefore, the customer will receive $23.00 in change.
    
    Length-Based Filtering:
    Reasoning Path 1: 5 steps (exceeds threshold)
    Reasoning Path 2: 3 steps (meets threshold)
    Reasoning Path 3: 3 steps (meets threshold)
    All reasoning paths have at least 3 steps, which meets our complexity threshold.
    
    Majority Vote Solution:
    All three reasoning paths arrived at the same answer: $23.00 in change.
    Therefore, based on the majority vote (3 out of 3), the customer will receive $23.00 in change.
    """),
    ("human", "{question}"),
    ("assistant", """
    I'll solve this problem using Complexity-based Prompting to leverage complex examples and multiple reasoning paths:
    
    Step 1: Complexity Analysis
    [Analysis of the question's complexity including length, required steps, and mathematical concepts involved]
    
    Step 2: Complex Exemplars
    [3-5 complex examples that demonstrate similar reasoning patterns with detailed solution paths]
    
    Step 3: Multiple Reasoning Paths
    [Generate 3 distinct approaches to solving the problem, each with step-by-step reasoning]
    
    Step 4: Length-Based Filtering
    [Evaluation of which reasoning chains meet the complexity threshold for consideration]
    
    Step 5: Majority Vote Solution
    [Final comprehensive solution based on the consensus among qualified reasoning chains]
    """)
])

In [5]:
# Fonction pour générer le prompt ThoT (node)
def generate_thot_node(state):
    question = state['messages'][-1].content  # Récupère la dernière question
    prompt_value = complexity_based_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    return {"messages": [response]}  # Ajoute la réponse comme un message

# Définition de l'état
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Création du graphe
graph_builder = StateGraph(State)
graph_builder.add_node("generate_thot", generate_thot_node)

# Configuration du graphe
graph_builder.set_entry_point("generate_thot")
graph_builder.add_edge("generate_thot", END)
graph = graph_builder.compile()

# Exemple d'utilisation
inputs = {"messages": [HumanMessage(content="Design an algorithm to find the kth smallest element in an unsorted array of n distinct integers in expected O(n) time complexity. Explain your approach, analyze its time and space complexity, and demonstrate how it works on the array [9, 3, 2, 7, 5, 1, 8, 6, 4] for k=4.")]}

# Affiche la question posée au début
print("Question posée:", inputs["messages"][0].content)
print("\nRéponse:")

for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_thot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)

Question posée: Design an algorithm to find the kth smallest element in an unsorted array of n distinct integers in expected O(n) time complexity. Explain your approach, analyze its time and space complexity, and demonstrate how it works on the array [9, 3, 2, 7, 5, 1, 8, 6, 4] for k=4.

Réponse:
Step 1: Problem complexity assessment:
The problem of finding the kth smallest element in an unsorted array of n distinct integers is a selection problem that can be solved efficiently. The complexity arises from the need to partition the array and recursively narrow down the search space. The expected time complexity is O(n) due to the use of the Quickselect algorithm, which is based on the partitioning method of QuickSort. The problem requires understanding of:
- Array manipulation
- Partitioning logic
- Recursive algorithms
This problem has moderate complexity, requiring several reasoning steps to implement and analyze the algorithm.

Step 2: Detailed reasoning chain:
To find the kth smalle